Downloading the Data

Importing Libraries

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
# stop_words = list(set(stopwords.words('english')))

In [ ]:
# stop_words

NameError: ignored

Reading train data

In [ ]:
train = pd.read_csv("hw1_train.csv")

FileNotFoundError: ignored

# New section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train

,ID,UTTERANCES,CORE RELATIONS
0,0,who plays luke on star wars new hope,movie.starring.actor movie.starring.character
1,1,show credits for the godfather,movie.starring.actor
2,2,who was the main actor in the exorcist,movie.starring.actor
3,3,find the female actress from the movie she's t...,movie.starring.actor actor.gender
4,4,who played dory on finding nemo,movie.starring.actor movie.starring.character
...,...,...,...
2307,2307,what was the revenue for toy story 3,movie.gross_revenue
2308,2308,dark knight revenue,movie.gross_revenue
2309,2309,how much did the dark night generate,movie.gross_revenue
2310,2310,can i see the lion king's revenue,movie.gross_revenue


In [ ]:
train.columns = ['ID','text','label']

In [ ]:
train.head()

,ID,text,label
0,0,who plays luke on star wars new hope,movie.starring.actor movie.starring.character
1,1,show credits for the godfather,movie.starring.actor
2,2,who was the main actor in the exorcist,movie.starring.actor
3,3,find the female actress from the movie she's t...,movie.starring.actor actor.gender
4,4,who played dory on finding nemo,movie.starring.actor movie.starring.character


checking the class distribution

In [ ]:
value_counts = train['label'].value_counts()

In [ ]:
value_counts

movie.directed_by                                        323
none                                                     318
movie.starring.actor                                     302
movie.rating                                             189
movie.produced_by                                        172
movie.initial_release_date                               142
movie.language                                           139
movie.country                                            136
movie.genre                                              101
movie.subjects                                            90
movie.production_companies                                84
movie.estimated_budget                                    78
movie.country movie.language                              72
movie.gross_revenue                                       28
movie.starring.actor movie.starring.character             19
movie.starring.actor movie.initial_release_date           18
movie.rating movie.initi

Checking for nans

In [ ]:
train = train.dropna()

In [ ]:
train.isna().sum()

ID       0
text     0
label    0
dtype: int64

In [ ]:
train

,ID,text,label
0,0,who plays luke on star wars new hope,movie.starring.actor movie.starring.character
1,1,show credits for the godfather,movie.starring.actor
2,2,who was the main actor in the exorcist,movie.starring.actor
3,3,find the female actress from the movie she's t...,movie.starring.actor actor.gender
4,4,who played dory on finding nemo,movie.starring.actor movie.starring.character
...,...,...,...
2307,2307,what was the revenue for toy story 3,movie.gross_revenue
2308,2308,dark knight revenue,movie.gross_revenue
2309,2309,how much did the dark night generate,movie.gross_revenue
2310,2310,can i see the lion king's revenue,movie.gross_revenue


train test split

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(train, test_size=0.20, random_state=42)

In [ ]:
train_df.shape

(1848, 3)

In [ ]:
test_df.shape

(463, 3)

In [ ]:
train_df.head(2)

# Feature Genearation

Using TF-IDF for vectorizing the data with N-Grams ranging from 1 to 3

In [ ]:
texts_train = list(train_df['text'].values)
vect_word = TfidfVectorizer(max_features=2500, analyzer='word', stop_words='english', ngram_range=(1,5), dtype=np.float32)
vect_word.fit(texts_train)
tfidf_complete_train = vect_word.transform(texts_train)

NameError: ignored

In [ ]:
tfidf_complete_train

<1848x2500 sparse matrix of type '<class 'numpy.float32'>'
	with 10413 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf_array_train = tfidf_complete_train.toarray()

In [ ]:
tfidf_array_train.shape

(1848, 2500)

In [ ]:
text_test = list(test_df['text'].values)
tfidf_complete_test = vect_word.transform(text_test).toarray()

In [ ]:
tfidf_complete_test.shape

(463, 2500)

In [ ]:
y= train_df['label'].values

#Modeling

Applying the first model Multinomial Naive Bayse

In [ ]:
# from sklearn.naive_bayes import MultinomialNB
# # from sklearn.linear_model import LogisticRegression
# # clf = LogisticRegression(random_state=0)
# clf = MultinomialNB(force_alpha=True)
# clf.fit(tfidf_array_train,y)


MultinomialNB(force_alpha=True)

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
clf.fit(tfidf_array_train, y)


Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('svc', SVC(gamma='auto'))])

In [ ]:
preds = clf.predict(tfidf_complete_test)

In [ ]:
preds

array(['movie.subjects', 'movie.directed_by', 'movie.country',
       'movie.starring.actor', 'movie.produced_by', 'movie.directed_by',
       'movie.produced_by', 'movie.produced_by', 'none',
       'movie.directed_by', 'movie.rating', 'movie.directed_by',
       'movie.directed_by', 'movie.estimated_budget',
       'movie.starring.actor', 'movie.directed_by', 'movie.rating',
       'movie.produced_by', 'movie.produced_by', 'movie.directed_by',
       'movie.rating', 'movie.language', 'none', 'movie.directed_by',
       'movie.language', 'movie.language', 'movie.produced_by',
       'movie.directed_by', 'movie.subjects', 'movie.language',
       'movie.language', 'movie.production_companies', 'movie.rating',
       'movie.rating', 'none', 'movie.starring.actor', 'movie.language',
       'movie.starring.actor', 'movie.starring.actor',
       'movie.directed_by', 'movie.rating', 'none',
       'movie.country movie.language', 'movie.starring.actor',
       'movie.directed_by', 'movie.rat

Classification Matrix for NB

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_df['label'], preds))

                                                       precision    recall  f1-score   support

                                        movie.country       0.95      0.65      0.77        31
                            movie.country movie.genre       0.00      0.00      0.00         3
                         movie.country movie.language       1.00      0.60      0.75        10
                           movie.country movie.rating       0.00      0.00      0.00         1
                                    movie.directed_by       0.68      0.82      0.74        67
         movie.directed_by movie.initial_release_date       0.00      0.00      0.00         1
                               movie.estimated_budget       1.00      0.48      0.65        23
                                          movie.genre       1.00      0.58      0.74        24
                           movie.genre movie.subjects       0.00      0.00      0.00         1
                                  movie.gross_rev

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# tfidf_complete_test.shape

In [ ]:
final_test = pd.read_csv("hw1_test.csv")

In [ ]:
final_test.shape

(981, 2)

In [ ]:
final_test.head()

,ID,UTTERANCES
0,0,star of thor
1,1,who is in the movie the campaign
2,2,list the cast of the movie the campaign
3,3,who was in twilight
4,4,who is in vulguria


In [ ]:
final_test.columns = ['ID','text']

In [ ]:
final_test.head(3)

,ID,UTTERANCES
0,0,star of thor
1,1,who is in the movie the campaign
2,2,list the cast of the movie the campaign


In [ ]:
text_test_final = list(final_test['text'].values)
tfidf_complete_test_final = vect_word.transform(text_test_final).toarray()

In [ ]:
tfidf_complete_test_final.shape

(981, 2500)

In [ ]:
preds_final = clf.predict(tfidf_complete_test_final)

In [ ]:
preds_final

In [ ]:
predictionssss = pd.DataFrame(zip(final_test['text'].values,preds_final),columns = ['text','predictions'])

In [ ]:
predictionssss.head()

,text,predictions
0,star of thor,movie.starring.actor
1,who is in the movie the campaign,none
2,list the cast of the movie the campaign,movie.starring.actor
3,who was in twilight,movie.directed_by
4,who is in vulguria,none


In [ ]:
predictionssss.to_csv("preds.csv",index = False)

In [ ]:
sample_submission = pd.read_csv('exampleSubmission.csv')

In [ ]:
sample_submission.head(3)

,ID,Core Relations
0,1,movie.initial_release_date movie.starring.actor
1,2,movie.initial_release_date movie.starring.actor
2,3,movie.initial_release_date movie.starring.actor


In [ ]:
sample_submission['Core Relations'] = list(preds_final)

In [ ]:
sample_submission.head(3)

,ID,Core Relations
0,1,movie.starring.actor
1,2,none
2,3,movie.starring.actor


In [ ]:
sample_submission.to_csv('submission1_SVC_TFIDF.csv',index=False)

Classification Matrix for SVC

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_df['label'], preds))

                                                       precision    recall  f1-score   support

                                        movie.country       0.95      0.65      0.77        31
                            movie.country movie.genre       0.00      0.00      0.00         3
                         movie.country movie.language       1.00      0.60      0.75        10
                           movie.country movie.rating       0.00      0.00      0.00         1
                                    movie.directed_by       0.68      0.82      0.74        67
         movie.directed_by movie.initial_release_date       0.00      0.00      0.00         1
                               movie.estimated_budget       1.00      0.48      0.65        23
                                          movie.genre       1.00      0.58      0.74        24
                           movie.genre movie.subjects       0.00      0.00      0.00         1
                                  movie.gross_rev

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Result: SVC Performs a little better